# Calculating Headways for CTA Buses - work in progress

NOTE: Headways for late night routes are not accurate yet - See the Needs Fixing section below.

### What This Does

- Calculates actual headways for each CTA bus stop on a given route, direction, service date, and start/end time. Actual headways are based on CTA Bus Tracker GPS data. 

- Generates summary stats for an entire route (min, max, mean, median, 25th percentile, and 75th percentile headways).

- Calculate Average Wait Times (AWT) - thanks to Sean MacMullan.


## In progress

- Calculate scheduled headways at each stop, making sure they make sense given different patterns that
run at different times of day for each route



## To Do Next

- Add summary stats to the stops geodataframe so they show up in the visualizations when you hover over a stop

- Investigate EWT calcs Sean found: https://www.trapezegroup.com.au/resources/infographic-how-to-calculate-excess-waiting-time/ 

- See what happens if you try to loop through many - or all - routes and compile a larger headways dataframe

## Notes

One bus route can be made up of several patterns.  Headways are calculated for all buses running the same direction on a given route at a particular stop, regardless which pattern the bus is on.   

Vehicle data comes from the Chi Hack Night Ghost Buses breakout team: https://ghostbuses.com/about
This provides location information in 5 minute intervals for every CTA bus.  It includes
data on which route (rt) and pattern (pid) the bus was running, along with the vehicle's distance along the pattern (pdist) and a timestamp.  

Pattern data comes from the CTA's API directly. This tells us which stops are found along
a given pattern and the distance along the pattern where each stop is located.

## Strategy for Actual Headways

Combining the datasets above, the general strategy is:

1. Turn vehicle data into intervals:  Time and distance are recorded at the start and end of each 5-minute interval.

2. For a given stop and pattern, find all intervals where a vehicle on that pattern reached or pased the stop.

3. Estimate the time each bus acutally reached the stop through interpolation.  The interval gives time and distance location along a given pattern before and after the bus arrived at the stop.  The CTA's pattern data tells us where the bus stop falls along the pattern.  Stop times are estimated assuming the vehicle travels a constant spaeed througout the interval.

4. Combine all stop times for buses running the same direction at a particular stop.

5. Calculate headways between buses based on stop times.

6. Calculate summary statistics on headways.

## Strategy for Scheduled Headways (Tentative)

1. Which patterns/shapes serve a given stop?

2. When are the earliest/latest times those patterns are in effect?

3. What are the headways within each timeframe?

## Needs Fixing

Address late night routes. 

- Some buses run past midnight, so the first bus of the calendar day appears just after midnight. 
- Many of these have no scheduled service in the early morning hours, but the unscheduled hours show up as a long headway.  And different directions of travel may begin and end service at different times. 
    - This code lets you choose a timeframe to analyze, but pinning down what ranges to use is tricky - need more work on this. 
    -
- Possible Approach?

    - DONE: Combine 2 days of data: the "main" date being looked at and the following date for any schedules that splill into overnight. 

    - How do we choose a range of time for each stop that doesn't include overnight out-of-service periods (which would show up as long headways)? Investigate the schedule data from CTA. Can we extract actual start/end times for a route/direction, adding a buffer of extra time on the end to capture buses finishing their routes at end of the schedule? See strategy for scheduled headways above.

In [1]:
import requests
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString
import datetime as dt
import numpy as np
from static_gtfs_analysis import *

In [2]:
# Get API key from the .env file
load_dotenv()
API_KEY = os.getenv('API_KEY')

# Get Actual Stop Times and Headways

### Functions

In [188]:
def get_chn_vehicles(date_string:str, start_timedelta_string:str='02:30', end_timedelta_string:str='26:30') -> pd.DataFrame:
    """Parameters:\n

    date_string in 'YYYY-MM-DD'format\n

    start_timedelta_string in 'hh:mm' format.  (optional. Default is '02:30')\n
    end_timedelta_string in 'hh:mm' format. (optional. Default is '26:30)' \n

    Data returned:\n

    Vehicle data scraped by the chn ghost bus team for all CTA buses running between the specified
    start and end times on the specified date.  Where an end time over 24 hours is specified, the data returned
    will extend into the following calendar date. The maximum valid 
    value for start_timedelta_string or end_timedelta_string is '23:59'\n
    
    Timedelta values start at midnight on day 1.  Example: If start_timedelta_string is '03:45' 
    and end_timedelta_sring is '25:07', then data returned is from 3:45 am on the requested date 
    through 1:07 am the following day. \n
    
    Data is returned in a pandas dataframe. Columns include vehicle id (vid), timestamp (tmstmp), 
    pattern id (pid), and distance along the pattern (pdist) for each vehicle at 5-minute intervals 
    throughout the requested time range on the requested calendar day.
    
    """

    day1 = pd.to_datetime(date_string, infer_datetime_format=True)
    day2 = day1 + pd.Timedelta(days=1)
    day2_string = day2.strftime('%Y-%m-%d')

    start_timedelta_string_expanded = start_timedelta_string + ':00'
    end_timedelta_string_expanded = end_timedelta_string + ':00'

    def get_vehicles_single_day(single_day_datestring):
        chn_data_source_single_day = f'https://chn-ghost-buses-public.s3.us-east-2.amazonaws.com/bus_full_day_data_v2/{single_day_datestring}.csv'
        vehicles_single_day = pd.read_csv(
        chn_data_source_single_day, dtype={
            'vid':'int',
            'tmstmp':'str',
            'lat':'float',
            'lon':'float',
            'hdg':'int',
            'pid':'int',
            'rt':'str',
            'pdist':'int',
            'des':'str',
            'dly':'bool',
            'tatripid':'str',
            'origatripno':'int',
            'tablockid':'str',
            'zone':'str',
            'scrape_file':'str',
            'data_hour':'int',
            'data_date':'str'
            }
        )

        vehicles_single_day['tmstmp'] = pd.to_datetime(vehicles_single_day['tmstmp'],infer_datetime_format=True)
    
        return vehicles_single_day

    df_day1_vehicles = get_vehicles_single_day(date_string)
    df_day2_vehicles = get_vehicles_single_day(day2_string)
    
    df_both_days_vehicles = pd.concat([df_day1_vehicles, df_day2_vehicles])

    # Filter for vehicles running between 2:50 am on day 1 and 2:50 am on day 2.
    # Note:  First tried 2:30, but caught one bus on the test day / route that had their
    # out of service time based on actual stop times from 3:42 am to 4:37 am.  
    # Then changed the window to 3:50am and found another bus stop with a bus
    # at 4:11 and another at 5:50.
    # another 

    # TODO
    # Possible to run quickly check every bus route and see if the longest headway is actually
    # an out of service time around 3am?

    service_day_start = day1+pd.Timedelta(start_timedelta_string_expanded)
    service_day_end = day1+pd.Timedelta(end_timedelta_string_expanded)

    df_vehicles = df_both_days_vehicles.loc[
        (service_day_start < df_both_days_vehicles['tmstmp'])
        & (df_both_days_vehicles['tmstmp'] <= service_day_end)]
 
    return df_vehicles


In [189]:
def get_patterns(vehicles:pd.DataFrame, rt:str) -> pd.DataFrame:
    '''This is a helper function.\n
    Parameters:\n
    vehicles is a dataframe obtained using get_chn_vehicles().\n
    rt is a route id as a string (for example, '55' for the 55 Garfield bus)\n
    Data returned:\n
    patterns data from the CTA's bus tracker API is returned in a dataframe. 
    It includes all pattern ids (pid) found in the in the vehicles data for the specified
    route. Columns include pattern id (pid) and points (pt).\n
    The pt data for each pattern is its own dataframe with information on every point
    along the pattern. It includes columns for sequence (seq), latitude (lat),
    longitude (lon), type of points (typ) where S indicates a bus stop,
    stop ID (stpid) for stop points, and distance along the pattern (pdist).
     '''
    

    df_output = pd.DataFrame()

    # filter vehicles to the specified route
    rt_vehicles = vehicles.loc[vehicles['rt'] == rt]

    # list pid values included in the route
    pid_list = list(rt_vehicles['pid'].unique())

    # convert pids to strings
    pid_list = [str(i) for i in pid_list]

    # split pid_list into chunks of 10 (limit of the API):
    start = 0
    end = len(pid_list)
    step = 10
    for i in range(start, end, step):
        pid_list_chunk = pid_list[i:i+step]
        pid_string = ','.join(pid_list_chunk)

        # get data from CTA's feed
        api_url = f'http://www.ctabustracker.com/bustime/api/v2/getpatterns?key={API_KEY}&pid={pid_string}&format=json'
        response = requests.get(api_url)
        patterns = response.json()

        # convert json to dataframe
        df_patterns = pd.DataFrame(patterns['bustime-response']['ptr'])

        # add to the output dataframe
        df_output = pd.concat([df_output, df_patterns])


    # convert pt column values to dataframes for each pattern containing that pattern's points
    df_output['pt'] = df_output['pt'].apply(lambda x: pd.DataFrame(x))
    
    return df_output


In [190]:

def get_pattern_linestrings(patterns:pd.DataFrame) -> gpd.GeoDataFrame:
    '''This is for future use and visualization - not neccessary to generate
    headway information.\n
    Paremeters:\n
    patterns is a dataframe obtained using get_patterns().\n
    Data returned:\n
    Pattern data is returned as a geodataframe wiht linestring geometry
    representing the path buses travel.'''

    df_patterns = patterns.copy()

    # Turn points into linestrings
    geometry_linestrings = []
    for p in df_patterns['pt']:
        p.sort_values('seq', inplace=True)
        linestring_points = list(zip(p['lon'],p['lat']))

        # generate linestring using all points
        linestring = LineString(linestring_points)
        geometry_linestrings.append(linestring)

    # Create a geodataframe for the patterns using the linestring geometry
    gdf_patterns = gpd.GeoDataFrame(df_patterns, geometry=geometry_linestrings).set_crs(epsg=4326)

    # Drop the original pt column
    gdf_patterns.drop(['pt'], axis=1, inplace=True)

    return gdf_patterns


In [191]:
def get_pattern_stops(patterns) -> gpd.GeoDataFrame:
        '''This is a helper function.\n
        Parameters:\n
        patterns is a dataframe obtained using get_patterns().\n
        Data returned:\n
        Bus stop data is returned as a geodataframe 
        with point geomtry, one point per bus stop on each pattern
        associated with a route.\n
        Note that stops serving multiple patterns will be listed multiple 
        times, once for each pattern with the seq and pdist values 
        specific to that pattern.'''

        # get patterns for the route
        df_patterns = patterns.copy()

        # set up a geodataframe to contain stops
        gdf_route_stops = gpd.GeoDataFrame()

        # consider the pid column (pattern ID) and the pt column (dataframe contaning
        # points along the pattern)
        for pid, pt in zip(df_patterns['pid'],df_patterns['pt']):
                # sort points sequentially
                pt.sort_values('seq', inplace=True)
                # add the pattern id to each point's data
                pt['pid']=pid
                # add the pattern direction to each point's data
                rtdir = df_patterns['rtdir'].loc[df_patterns['pid'] == pid].tolist()[0]
                pt['rtdir'] = rtdir
                # filter to only show stop points
                stops = pt[pt['typ']=='S']
                # zip lat/lon data to get coordinate pairs
                coords = list(zip(stops['lon'],stops['lat']))
                # turn coordinates into point geometry
                geometry = [Point(c) for c in coords]
                # generate a geodataframe for the stops in this pattern
                gdf_pattern_stops = gpd.GeoDataFrame(stops,geometry=geometry).set_crs(epsg=4326)
                # add this pattern's stops to the dataframe containing all stops on the route
                gdf_route_stops = pd.concat([gdf_route_stops, gdf_pattern_stops])

        return gdf_route_stops


In [192]:
def get_vehicle_intervals(vehicles:pd.DataFrame, rt:str) -> pd.DataFrame:

    '''This is a helper function.\n
    Parameters:\n
    vehicles is a dataframe obtained using get_chn_vehicles().\n
    Data returned:\n
    Intervals are returned as a dataframe, with each row representing
    an interval between two points in time and space for one vehicle. 
    Columns are added to the vehicles data for each interval's 
    start time, end time, start pdist, and end pdist.'''

    df_vehicles = vehicles.copy()

    # filter to the specified route
    df_vehicles = df_vehicles.loc[df_vehicles['rt'] == rt]

    # Set up dataframe to contain final fomratted data
    df_output = pd.DataFrame()

    # End time for each interval as a timestamp
    # df_vehicles['end_time'] = pd.to_datetime(df_vehicles['tmstmp'],infer_datetime_format=True)
    df_vehicles['end_time'] = df_vehicles['tmstmp']
    vid_list = df_vehicles['vid'].unique().tolist()

    # End location for each interval
    df_vehicles['end_pdist'] = df_vehicles['pdist']

    for vid in vid_list:

        # pare data down to a single vehicle
        df_vehicle = df_vehicles.loc[df_vehicles['vid'] == vid]

        # handle each pattern separately
        pid_list = df_vehicle['pid'].unique().tolist()
        for p in pid_list:
            df_vehicle_pattern = df_vehicle.loc[df_vehicle['pid']==p].copy()
            # sort by time (it should be sorted already, but just in case)
            df_vehicle_pattern.sort_values(by=['end_time'], inplace=True)

            # Create a start time based on the previous tinmestamp
            end_times = df_vehicle_pattern['end_time'].tolist()
            start_times = np.roll(end_times,shift=1)
            df_vehicle_pattern['start_time'] = start_times

            # Create a start pattern distance based on the previous pdist
            end_distances = df_vehicle_pattern['end_pdist'].tolist()
            start_distances = np.roll(end_distances,shift=1)
            df_vehicle_pattern['start_pdist'] = start_distances

            # Remove the first interval since we don't have real start
            # time or location data for it
            df_vehicle_pattern = df_vehicle_pattern.iloc[1:]

            # add data to the full output dataframe
            df_output = pd.concat([df_output, df_vehicle_pattern])

    return df_output



In [193]:
        # Interpolate estimated times the bus arrived at a stop
        def interpolate_stop_time(
            stop_pdist:int, 
            start_time:pd.Timestamp, 
            end_time:pd.Timestamp, 
            start_pdist:int, 
            end_pdist:int
            ) -> pd.Timestamp:

            '''This is a helper function.\n
            Parameters:\n
            stop_pdist is an integer distance along a pattern to a given bus stop.\n
            start_time and end_time are timestamps for the beginning and end of an interval.\n
            start_pdist and end_pdist are integer distances along a pattern at the beginning and
            end of an interval.\n
            Data returned:\n
            timestamp for the estimated time a vehicle reached a stop, assuming it
            traveled a constant speed from start to end of teh interval
            '''

            # How far into the interval distance is the bus stop?
            # stop distance from beginning of interval / full interval distance
            dist_ratio = (stop_pdist-start_pdist)/(end_pdist-start_pdist)

            # estimated bus stop time, assuming it traveled at a steady
            # speed throughout the interval
            est_stop_time = start_time + (end_time - start_time)*dist_ratio

            # round estimated stop time to the nearest minute
            est_stop_time = est_stop_time.round(freq='T')

            return est_stop_time

In [194]:
def get_stoptimes(rt:str, vehicles:pd.DataFrame) -> pd.DataFrame:

    '''This is a helper function.\n
    Parameters:\n
    vehicles is a dataframe obtained using get_chn_vehicles().\n
    rt is a route id as a string (for example, '55' for the 55 Garfield bus)\n
    Data returned:\n
    Columns are added to the vehicles dataframe indicating the start and end time
    and the start and end distances along a pattern for each interval where a bus
    passed a stop (start_time, end_time, start_pdist, end_pdist). The estimated time 
    each bus actually arrived at the stop (est_stop_time) is also added.\n
    The dataframe returned covers all buses at all stops on the specified route'''
 
    # set up a dataframe to contain the output data
    df_output = pd.DataFrame()

    # turn vehicle data into intervals between vehicles
    vehicle_intervals = get_vehicle_intervals(vehicles, rt)

    # get pattern data from the CTA
    df_patterns = get_patterns(vehicles, rt)

    # get all stops on this route, including all patterns
    gdf_stops = get_pattern_stops(df_patterns)

    # Consider each combination of stop and pattern
    for stpid, pid, rtdir in list(zip(gdf_stops['stpid'],gdf_stops['pid'], gdf_stops['rtdir'])):

        # get a single stop on a single pattern
        gdf_this_stop_pattern = gdf_stops.loc[(gdf_stops['stpid'] == stpid) & (gdf_stops['pid'] == pid)]
        if len(gdf_this_stop_pattern) == 0:
            continue
            
        # Find the bus stop's distance along the pattern
        pdist_this_stop = gdf_this_stop_pattern['pdist'].tolist()[0]

        # find the intervals that are on this pattern
        df_this_pattern_intervals = vehicle_intervals.loc[vehicle_intervals['pid'] == pid]
        if len(df_this_pattern_intervals) == 0:
            continue

        # Filter for intervals that start ahead of the stop location and end at or beyond the stop
        def filter_intervals(stop_dist:int, start_pdist:int, end_pdist:int):
            return (start_pdist < stop_dist) & (end_pdist >= stop_dist)
    
        # Create filter for the intervals we're working on
        interval_filter = df_this_pattern_intervals.apply(
            lambda x: filter_intervals(pdist_this_stop, x['start_pdist'], x['end_pdist']), axis=1
            )
        
        # apply the filter
        df_this_pattern_stop_intervals = df_this_pattern_intervals.loc[interval_filter]
        if len(df_this_pattern_stop_intervals) == 0:
            continue

        # Add stpid, pdist, and rtdir to the data
        df_this_pattern_stop_intervals['stpid'] = stpid
        df_this_pattern_stop_intervals['stop_pdist'] = int(pdist_this_stop)
        df_this_pattern_stop_intervals['rtdir'] = rtdir


        # Estimate time each bus passed the stop (interpolated based on data at start and
        # end of the interval)
        df_this_pattern_stop_intervals['est_stop_time'] = df_this_pattern_stop_intervals.apply(
            lambda x: interpolate_stop_time(
                pdist_this_stop, 
                x['start_time'], 
                x['end_time'], 
                x['start_pdist'], 
                x['end_pdist']), axis=1
            )

        # Add the intervals with stop times to the full output dataframe
        df_output = pd.concat([df_output, df_this_pattern_stop_intervals])

    return df_output


In [195]:

def get_headways(rt:str, vehicles:pd.DataFrame) -> pd.DataFrame:

        '''Parameters:\n
        vehicles is a dataframe obtained using get_chn_vehicles().\n
        rt is a route id as a string (for example, '55' for the 55 Garfield bus)\n
        Data returned:\n
        Columns are added to the vehicles dataframe indicating the start and end time
        and the start and end distances along a pattern for each interval where a bus
        passed a stop (start_time, end_time, start_pdist, end_pdist), the estimated time 
        each bus actually arrived at the stop (est_stop_time), and headway between each 
        bus (est_headway).  Direction of travel (rtdir) and stop id (stpid)
        are also included.\n
        The dataframe returned covers all buses at all stops on the specified route'''

        df_output = pd.DataFrame()

        # Times buses stopped at each stop on the route
        df_stoptimes = get_stoptimes(rt, vehicles).copy()

        # consider all buses stopping at a given stop moving int he same direction
        for stpid, rtdir in set(zip(df_stoptimes['stpid'], df_stoptimes['rtdir'])):

                # filter data
                df_stop_direction = df_stoptimes.loc[(df_stoptimes['stpid'] == stpid) & (df_stoptimes['rtdir'] == rtdir)]

                # Sort chronologically
                df_stop_direction.sort_values(by='est_stop_time',ascending=True, inplace=True)

                # list stop times in chronological order
                stop_times = df_stop_direction['est_stop_time'].tolist()

                # calculate previous stop time for each line
                prev_stop_times = np.roll(stop_times,1)
                df_stop_direction['previous_stop_time'] = prev_stop_times

                # calculate headway
                df_stop_direction['est_headway'] = (
                df_stop_direction['est_stop_time'] - df_stop_direction['previous_stop_time']
                )
                df_stop_direction['est_headway'] = df_stop_direction['est_headway']

                # drop previous stop time column, no longer needed
                df_stop_direction = df_stop_direction.drop('previous_stop_time', axis=1)

                # Remove headway from the first bus in the dataset since we don't have the 
                # previous bus to compare with
                df_stop_direction['est_headway'].iloc[0] = None
                
                df_output = pd.concat([df_output, df_stop_direction])

        return df_output



In [196]:
def get_headway_stats(headways:pd.DataFrame, headway_column_name:LineString) -> dict:
    '''Parameters:\n
    headways is a dataframe obtained using get_headways() or get_scheduled_headways().\n
    headway_column_name is the name of the column containing headways:  'est_headway' if these
    are based on actual bus times using get_headways() or 'scheduled_headway' if these are based
    on GTFS schedules using get_Scheduled_headways()\n
 
    Data returned:\n
    Statisics on the headways are returned as a dictionary.'''
    est_headways = headways[headway_column_name]
    stats = {
        'mean':est_headways.mean(),
        'max':est_headways.max(),
        'min':est_headways.min(),
        '25th_percentile':est_headways.quantile(0.25), # 25th percentile
        'median':est_headways.median(), # 50th percentile
        '75th_percentile':est_headways.quantile(0.75)
    }
    return stats




In [197]:
def get_average_wait_time(headways:pd.DataFrame) -> pd.DataFrame:
    '''Parameters:\n
    headways is a dataframe obtained using get_headways().\n
    Data returned:\n
    Average wait time (AWT) value by stop ID.'''

    # AWT = SUM(D^2)/2T, where D = the duration between arrivals and T = the timeframe duration.
    # When D=T, this simplifies to AWT = D/T
    stops = pd.DataFrame()
    stops['stpid'] = headways['stpid'].unique()
    
    AWTs = []
    mean = []
    for stop in stops['stpid']:
        stop_visits = headways[headways['stpid'] == stop]
        
        start = stop_visits['start_time'].min()
        end = stop_visits['end_time'].max()
        timeframe_duration = pd.Timedelta(end-start).seconds/60.0

        headway_minutes = stop_visits['est_headway'].dt.total_seconds()/60
        AWT = ((headway_minutes)**2).sum()/(2*timeframe_duration)
        AWTs.append(AWT)

        mean.append(headway_minutes.mean())

    stops['AWT'] = AWTs
    stops['mean_headway'] = mean
    return stops


## Try it out: Get Actual Stop Times and Headways

In [198]:
vehicles = get_chn_vehicles('2023-01-11')

In [199]:
vehicles

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,tatripid,origtatripno,tablockid,zone,scrape_file,data_time,data_hour,data_date
3763,7973,2023-01-11 02:32:00,41.822785,-87.606773,1,2642,4,Illinois Center,15439,False,10000881,238694398,N4 -792,NaN,bus_data/2023-01-11/02:32:56.json,2023-01-11 02:32:00,2,2023-01-11
3764,7936,2023-01-11 02:32:00,41.882610,-87.627747,5,2642,4,Illinois Center,48361,False,10000874,238694475,N4 -791,NaN,bus_data/2023-01-11/02:32:56.json,2023-01-11 02:32:00,2,2023-01-11
3765,1086,2023-01-11 02:32:00,41.722362,-87.594824,179,12428,5,69th Red Line,10695,False,10003771,238664228,N95 -192,NaN,bus_data/2023-01-11/02:32:56.json,2023-01-11 02:32:00,2,2023-01-11
3766,1074,2023-01-11 02:32:00,41.773212,-87.598861,268,12428,5,69th Red Line,55633,False,10003770,238664261,N95 -193,NaN,bus_data/2023-01-11/02:32:56.json,2023-01-11 02:32:00,2,2023-01-11
3767,1177,2023-01-11 02:32:00,41.758879,-87.574355,89,12429,5,95th Red Line,21431,False,10003764,238664195,N95 -191,NaN,bus_data/2023-01-11/02:32:56.json,2023-01-11 02:32:00,2,2023-01-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3883,8365,2023-01-12 02:27:00,41.968843,-87.669590,179,16550,81,Wilson/Marine,27267,False,88348720,238703287,N81 -491,NaN,bus_data/2023-01-12/02:27:56.json,2023-01-12 02:27:00,2,2023-01-12
3884,8344,2023-01-12 02:27:00,41.968449,-87.707863,278,16551,81,Jefferson Park Blue Line,21419,False,88348718,238702167,81 -404,NaN,bus_data/2023-01-12/02:27:56.json,2023-01-12 02:27:00,2,2023-01-12
3885,8394,2023-01-12 02:27:00,41.969240,-87.760502,311,16551,81,Jefferson Park Blue Line,37055,True,88348719,238703274,N81 -492,NaN,bus_data/2023-01-12/02:27:56.json,2023-01-12 02:27:00,2,2023-01-12
3886,7993,2023-01-12 02:27:00,41.735798,-87.672447,270,5572,87,Western,13174,False,1082087,238695537,N87 -791,NaN,bus_data/2023-01-12/02:27:56.json,2023-01-12 02:27:00,2,2023-01-12


In [200]:
%%capture --no-display 
# turn off warnings

# Check out the 90 Harlem bus
headways = get_headways('90',vehicles)


In [201]:
headways.sort_values('est_headway', ascending=False)

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,...,data_date,end_time,end_pdist,start_time,start_pdist,stpid,stop_pdist,rtdir,est_stop_time,est_headway
168006,1510,2023-01-11 23:17:00,41.982048,-87.808205,28,2944,90,Harlem Blue Line,37723,False,...,2023-01-11,2023-01-11 23:17:00,37723,2023-01-11 23:12:00,30851,11827,37291,Northbound,2023-01-11 23:17:00,0 days 02:40:00
28661,8410,2023-01-11 07:47:00,41.981379,-87.807760,37,2944,90,Harlem Blue Line,37370,False,...,2023-01-11,2023-01-11 07:47:00,37370,2023-01-11 07:42:00,36422,11827,37291,Northbound,2023-01-11 07:47:00,0 days 02:20:00
85678,1396,2023-01-11 13:52:00,41.982210,-87.808134,37,2944,90,Harlem Blue Line,37853,False,...,2023-01-11,2023-01-11 13:52:00,37853,2023-01-11 13:47:00,35958,11827,37291,Northbound,2023-01-11 13:51:00,0 days 02:06:00
148178,1053,2023-01-11 19:32:00,41.981831,-87.808449,52,2944,90,Harlem Blue Line,37690,False,...,2023-01-11,2023-01-11 19:32:00,37690,2023-01-11 19:27:00,33477,11827,37291,Northbound,2023-01-11 19:32:00,0 days 02:05:00
66227,8297,2023-01-11 11:32:00,41.969489,-87.807152,182,5917,90,Harlem Green Line,5470,False,...,2023-01-11,2023-01-11 11:32:00,5470,2023-01-11 11:27:00,0,8535,962,Southbound,2023-01-11 11:28:00,0 days 01:56:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8264,1396,2023-01-11 05:22:00,41.900096,-87.805452,359,2944,90,Harlem Blue Line,5669,False,...,2023-01-11,2023-01-11 05:22:00,5669,2023-01-11 05:17:00,0,16063,448,Northbound,2023-01-11 05:17:00,NaT
8265,8297,2023-01-11 05:22:00,41.966949,-87.807053,0,2944,90,Harlem Blue Line,31434,False,...,2023-01-11,2023-01-11 05:22:00,31434,2023-01-11 05:17:00,26191,11816,29732,Northbound,2023-01-11 05:20:00,NaT
5806,8297,2023-01-11 04:37:00,41.935157,-87.806633,181,5917,90,Harlem Green Line,19084,False,...,2023-01-11,2023-01-11 04:37:00,19084,2023-01-11 04:32:00,13456,11778,14399,Southbound,2023-01-11 04:33:00,NaT
8642,8297,2023-01-11 05:27:00,41.981334,-87.807564,32,2944,90,Harlem Blue Line,37314,False,...,2023-01-11,2023-01-11 05:27:00,37314,2023-01-11 05:22:00,31434,11824,35185,Northbound,2023-01-11 05:25:00,NaT


In [202]:
headway_stats = get_headway_stats(headways, 'est_headway')
headway_stats

{'mean': Timedelta('0 days 00:26:24.657534246'),
 'max': Timedelta('0 days 02:40:00'),
 'min': Timedelta('0 days 00:06:00'),
 '25th_percentile': Timedelta('0 days 00:21:00'),
 'median': Timedelta('0 days 00:25:00'),
 '75th_percentile': Timedelta('0 days 00:30:00')}

In [203]:
AWTs = get_average_wait_time(headways)
AWTs

,stpid,AWT,mean_headway
0,15712,13.193665,25.000000
1,9740,14.195928,26.119048
2,11774,13.538636,25.511628
3,15702,13.910138,26.317073
4,11776,13.468326,25.511628
...,...,...,...
104,16063,35.905069,54.000000
105,11816,13.753846,24.954545
106,11778,13.560455,25.511628
107,11824,20.293182,32.264706


In [204]:
# Visualize routes and stops for the 55 bus (no headway info included in visualization yet)
patterns = get_patterns(vehicles, '55')
route = get_pattern_linestrings(patterns)
stops = get_pattern_stops(patterns)

# m = route.explore(color='blue', tiles='CartoDB_positron')
# stops.explore(m=m, color='red')


# Get Scheduled Stop Times and Headways


In [3]:
# Use Laurie's code to get gtfs feed data

gtfs_feed = download_extract_format('20230105')

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
Loading shapes.txt: 100%|██████████| 7/7 [00:04<00:00,  1.52it/s]


In [4]:
# Get stop data from gtfs feed
df_stops = gtfs_feed.stops

# Eliminate stops that don't have public facing stop codes
df_stops = df_stops.loc[df_stops['stop_code'].notnull()]

df_stops

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding
0,1,1,Jackson & Austin Terminal,"Jackson & Austin Terminal, Northeastbound, Bus...",41.87632184,-87.77410482,0,NaN,1
1,2,2,5900 W Jackson,"5900 W Jackson, Eastbound, Southside of the St...",41.87706679,-87.77131794,0,NaN,1
2,4,4,5700 W Jackson,"5700 W Jackson, Eastbound, Southside of the St...",41.87702418,-87.76745055,0,NaN,1
3,6,6,Jackson & Lotus,"Jackson & Lotus, Eastbound, Southeast Corner",41.876513,-87.761446,0,NaN,1
4,7,7,5351 W Jackson,"5351 W Jackson, Eastbound, Southside of the St...",41.87655197,-87.75892544,0,NaN,1
...,...,...,...,...,...,...,...,...,...
10763,18644,18644,Grand & Rockwell,"Grand & Rockwell, Eastbound, Southside of the ...",41.89215007,-87.69141601,0,NaN,1
10764,18645,18645,Grand & Western,"Grand & Western, Eastbound, Southeast Corner",41.89058595,-87.68636489,0,NaN,1
10765,18646,18646,Grand & Oakley,"Grand & Oakley, Eastbound, Southeast Corner",41.89063726,-87.68397303,0,NaN,1
11215,999990,999990,Rosemont Blue Line,Rosemont Blue Line Station Terminal,41.9840004,-87.860325,0,NaN,1


In [25]:
def get_scheduled_headways(
        route_id:str, 
        service_date_string:str, 
        start_timedelta_string:str='02:30', 
        end_timedelta_string:str='26:30') -> pd.DataFrame:

        '''Parameters:\n
        route_id is a route id as a string (for example, '55' for the 55 Garfield bus)\n
        start_date_string and end_date_string are in 'YYYY-MM-DD'format\n

        start_timedelta_string in 'hh:mm' format.  (optional. Default is '02:30')\n
        end_timedelta_string in 'hh:mm' format. (optional. Default is '26:30)'\n
        start and end timedeltas are relative to midnight on the service date.
        So a start_timedelta of '06:46' would be 6:46am on the service date, 
        while an end_timedelta of '26:02' would be 2:02 am the next day.  \n

        # Data returned:\n
        # dataframe containing scheduled trip information including shceudled headways
        # for all bus stops running on the specified route between the times specified
        # in the start_timedelta_string and end_timedelta_string. Default timeframe
        # if no start/end times are specified is 2:30 am on the specified date to 2:30 am
        # on the following date.\n
        '''

        # Create dataframe to contain headway data
        df_output = pd.DataFrame()

        def string_to_datetime(date_string):
                year = int(date_string[:4])
                month = int(date_string[5:7])
                day = int(date_string[8:])
                return pendulum.datetime(year, month, day)

        # Capture service date plus or minus one day.  Service that
        # started the previous day neds to be captured at each stop,
        # plus service that spills into the next day for late night routes.
        service_date = string_to_datetime(service_date_string)
        start_date = service_date - dt.timedelta(days=1)
        end_date = service_date + dt.timedelta(days=1)

        # Add seconds to the timedelta strings and convert to timedeltas
        start_timedelta = pd.to_timedelta(start_timedelta_string+':00')
        end_timedelta = pd.to_timedelta(end_timedelta_string+':00')
        
        start_time = service_date + start_timedelta
        end_time = service_date + end_timedelta

        print(start_time)
        print(end_time)

        # Get trip summary for service date plus/minus a day using chn-ghost-buses make_trip_summary() function
        trip_summary = make_trip_summary(gtfs_feed, start_date, end_date)

        # Turn arrival times into timestamps
        trip_summary['arrival_time'] =  pd.to_timedelta(trip_summary['arrival_time'])
        trip_summary['arrival_time'] += trip_summary['raw_date']

        # filter to the specified route
        trip_summary = trip_summary.loc[trip_summary['route_id'] == route_id]

        # Filter to the specified start/end times
        trip_summary = trip_summary.loc[trip_summary['arrival_time'] >= start_time]
        trip_summary = trip_summary.loc[trip_summary['arrival_time'] < end_time]


        ############## Edits start here ##################
        # What patterns serve a given stop?
        


        # consider all buses stopping at a given stop moving in the same direction
        # (Is direction necessary?  Are there any cases where a stop serves two directions?)
        for stop_id, direction in set(zip(trip_summary['stop_id'], trip_summary['direction'])):

                # filter data
                trip_summary_stop_direction = trip_summary.loc[
                        (trip_summary['stop_id'] == stop_id) 
                        & (trip_summary['direction'] == direction)
                        ]

                # Sort chronologically
                trip_summary_stop_direction.sort_values(by='arrival_time',ascending=True, inplace=True)

                # list stop times in chronological order
                stop_times = trip_summary_stop_direction['arrival_time'].tolist()

                # calculate previous stop times
                prev_arrival_times = np.roll(stop_times,1)
                trip_summary_stop_direction['previous_arrival_time'] = prev_arrival_times

                # calculate headway
                trip_summary_stop_direction['scheduled_headway'] = (
                        trip_summary_stop_direction['arrival_time'] - trip_summary_stop_direction['previous_arrival_time']
                        )

                # drop previous arrrival time column, no longer needed
                trip_summary_stop_direction = trip_summary_stop_direction.drop('previous_arrival_time', axis=1)

                # Remove headway from the first bus in the dataset since we don't have the 
                # previous bus to compare with
                trip_summary_stop_direction['scheduled_headway'].iloc[0] = None
                
                # Add
                df_output = pd.concat([df_output, trip_summary_stop_direction])

        return df_output

# Scratch

In [5]:
%%capture --no-display
# turn off warnings

######### Scratch
gtfs_version_id = '20230105'
route_id = '55'
service_date_string = '2023-01-09'
# start_timedelta_string ='02:30'
# end_timedelta_string = '26:30'

gtfs_feed = download_extract_format(gtfs_version_id)



INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
INFO:root:stops.txt loaded
INFO:root:stop_times.txt loaded
INFO:root:routes.txt loaded
INFO:root:trips.txt loaded
INFO:root:calendar.txt loaded
INFO:root:calendar_dates.txt loaded
INFO:root:shapes.txt loaded


In [17]:
def string_to_datetime(date_string):
        year = int(date_string[:4])
        month = int(date_string[5:7])
        day = int(date_string[8:])
        return pendulum.datetime(year, month, day)


In [18]:
############
# Revised:  Capture one service day of data
############


# Capture service date 
service_date = string_to_datetime(service_date_string)

# # Add seconds to the timedelta strings and convert to timedeltas
# start_timedelta = pd.to_timedelta(start_timedelta_string+':00')
# end_timedelta = pd.to_timedelta(end_timedelta_string+':00')

# start_time = service_date + start_timedelta
# end_time = service_date + end_timedelta

# print(start_time)
# print(end_time)


# Get trip summary for service date using chn-ghost-buses make_trip_summary() function
trip_summary = make_trip_summary(gtfs_feed, service_date, service_date)

# Turn arrival times into timestamps
trip_summary['stop_time'] =  trip_summary['raw_date'] + pd.to_timedelta(trip_summary['arrival_time'])

# filter to the specified route
trip_summary = trip_summary.loc[trip_summary['route_id'] == route_id]

# # Filter to the specified start/end times
# trip_summary = trip_summary.loc[trip_summary['arrival_time'] >= start_time]
# trip_summary = trip_summary.loc[trip_summary['arrival_time'] < end_time]

trip_summary

# scheduled_headways = get_scheduled_headways(
#     route_id, 
#     service_date_string, 
#     start_timedelta_string, 
#     end_timedelta_string
#     )


,route_id,service_id,trip_id,direction_id,block_id,shape_id,direction,wheelchair_accessible,schd_trip_id,raw_date,...,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,headway_secs,stop_time
975487,55,65301,6530000010020,0,653000003086,65301295,West,1,10020,2023-01-09 00:00:00+00:00,...,04:37:30,10603,1,St Louis,0,0,4,4,NaN,2023-01-09 04:37:30+00:00
975524,55,65301,6530000057020,0,653000003137,65305425,West,1,57020,2023-01-09 00:00:00+00:00,...,16:10:00,10565,1,Midway Orange Line,0,0,16,16,NaN,2023-01-09 16:10:00+00:00
975525,55,65301,6530000057020,0,653000003137,65305425,West,1,57020,2023-01-09 00:00:00+00:00,...,17:00:25,15366,63,Midway Orange Line,0,45089,17,17,32.0,2023-01-09 17:00:25+00:00
975598,55,65301,6530000586020,0,653000003122,65305425,West,1,586020,2023-01-09 00:00:00+00:00,...,23:42:30,10565,1,Midway Orange Line,0,0,23,23,NaN,2023-01-09 23:42:30+00:00
975599,55,65301,6530000586020,0,653000003122,65305425,West,1,586020,2023-01-09 00:00:00+00:00,...,24:00:05,10597,29,Midway Orange Line,0,21664,0,0,30.0,2023-01-10 00:00:05+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996217,55,65315,6530040471020,0,653000001185,65305425,West,1,40471020,2023-01-09 00:00:00+00:00,...,01:00:18,10628,60,Midway Orange Line,0,43278,1,1,1871.0,2023-01-09 01:00:18+00:00
996232,55,65315,6530042106020,0,653000001186,65301293,West,1,42106020,2023-01-09 00:00:00+00:00,...,02:40:00,10565,1,St Louis,0,0,2,2,NaN,2023-01-09 02:40:00+00:00
996233,55,65315,6530042106020,0,653000001186,65301293,West,1,42106020,2023-01-09 00:00:00+00:00,...,03:00:17,10608,40,St Louis,0,29169,3,3,1078.0,2023-01-09 03:00:17+00:00
996248,55,65315,6530042199010,0,653000003768,65305425,West,1,42199010,2023-01-09 00:00:00+00:00,...,00:02:30,10565,1,Midway Orange Line,0,0,0,0,NaN,2023-01-09 00:02:30+00:00


In [8]:
# For a given route and stop, what service ID / block ID combos are found?


In [9]:

# # Change data type so it can be merged with df_stops
# trip_summary['stop_id'] = trip_summary['stop_id'].astype(int)

# Add stop data to the trip summary.  Inner join since the stops data is already 
# narrowed down to stops with public facing stop codes.
trip_summary = trip_summary.merge(df_stops, on='stop_id', how='inner')

trip_summary

,route_id,service_id,trip_id,direction_id,block_id,shape_id,direction,wheelchair_accessible,schd_trip_id,raw_date,...,departure_hour,stop_time,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding
0,55,65301,6530000010020,0,653000003086,65301295,West,1,10020,2023-01-09 00:00:00+00:00,...,4,2023-01-09 04:37:30+00:00,10603,Garfield & Ashland,"Garfield & Ashland, Westbound, Northwest Corner",41.79407388,-87.6648838,0,NaN,1
1,55,65301,6530001696020,0,653000003152,65301295,West,1,1696020,2023-01-09 00:00:00+00:00,...,5,2023-01-09 05:11:30+00:00,10603,Garfield & Ashland,"Garfield & Ashland, Westbound, Northwest Corner",41.79407388,-87.6648838,0,NaN,1
2,55,65301,6530012872020,0,653000003134,65301295,West,1,12872020,2023-01-09 00:00:00+00:00,...,7,2023-01-09 07:00:30+00:00,10603,Garfield & Ashland,"Garfield & Ashland, Westbound, Northwest Corner",41.79407388,-87.6648838,0,NaN,1
3,55,65301,6530023086020,0,653000003140,65305425,West,1,23086020,2023-01-09 00:00:00+00:00,...,5,2023-01-09 05:00:37+00:00,10603,Garfield & Ashland,"Garfield & Ashland, Westbound, Northwest Corner",41.79407388,-87.6648838,0,NaN,1
4,55,65301,6530023418020,0,653000002878,65301295,West,1,23418020,2023-01-09 00:00:00+00:00,...,6,2023-01-09 06:42:30+00:00,10603,Garfield & Ashland,"Garfield & Ashland, Westbound, Northwest Corner",41.79407388,-87.6648838,0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,55,65302,6530016074030,0,653000001118,65305426,West,1,16074030,2023-01-09 00:00:00+00:00,...,20,2023-01-09 20:00:14+00:00,10608,Garfield & Hoyne,"Garfield & Hoyne, Westbound, Northwest Corner",41.79395538,-87.67688782,0,NaN,1
427,55,65315,6530042106020,0,653000001186,65301293,West,1,42106020,2023-01-09 00:00:00+00:00,...,3,2023-01-09 03:00:17+00:00,10608,Garfield & Hoyne,"Garfield & Hoyne, Westbound, Northwest Corner",41.79395538,-87.67688782,0,NaN,1
428,55,65302,6530019731030,1,653000001117,65305424,East,1,19731030,2023-01-09 00:00:00+00:00,...,20,2023-01-09 20:00:01+00:00,17563,55th Street & Kenwood,"55th Street & Kenwood, Eastbound, Southeast Co...",41.79503478,-87.59301755,0,NaN,1
429,55,65302,6530038836020,1,653000001284,65303751,East,1,38836020,2023-01-09 00:00:00+00:00,...,15,2023-01-09 15:10:00+00:00,10538,Garfield & Lowe,"Garfield & Lowe, Eastbound, Southeast Corner",41.7939632,-87.64105011,0,NaN,1


In [28]:

# For a given stop, find the service ids that use it
stop_id = '10608'

service_ids = trip_summary.loc[trip_summary['stop_id'] == stop_id]['service_id'].unique().tolist()

service_ids


['65302', '65315']

In [29]:
# for a given servie id, find the earliest and latest bus.

# For each service, list all buses arriving at the stop in order.

for service_id in service_ids:

    trip_summary_test = trip_summary.loc[trip_summary['stop_id'] == stop_id].loc[trip_summary['service_id'] == service_id]

    service_start = min(trip_summary_test['arrival_time'])
    service_end = max(trip_summary_test['arrival_time'])

    print(f'Service id {service_id} starts at {service_start} and ends at {service_end}.')
    ##########
    ### Need to identify service day using the service data here, not by a time we feed into 
    ### the function.  Split day into service hours (as many as there are) and analyze the 
    ### headways for each separate servide on that day individually.'

    ### Summarize route - stop - service ID - times service is in effect - headways
    ###########
    


Service id 65302 starts at 0 days 20:00:14 and ends at 0 days 20:00:14.
Service id 65315 starts at 0 days 03:00:17 and ends at 0 days 03:00:17.


In [24]:



df_output = pd.DataFrame()

for stop_id, service_id in set(zip(trip_summary['stop_id'], trip_summary['service_id'])):

            # filter data
            trip_summary_stop_service = trip_summary.loc[
                    (trip_summary['stop_id'] == stop_id) 
                    & (trip_summary['service_id'] == service_id)
                    ]

            # Sort chronologically
            trip_summary_stop_service.sort_values(by='stop_time',ascending=True, inplace=True)

            # list stop times in chronological order
            stop_times = trip_summary_stop_service['stop_time'].tolist()

            # calculate previous stop times
            prev_stop_times = np.roll(stop_times,1)
            trip_summary_stop_service['previous_stop_time'] = prev_stop_times

            # calculate headway
            trip_summary_stop_service['scheduled_headway'] = (
                    trip_summary_stop_service['stop_time'] - trip_summary_stop_service['previous_stop_time']
                    )

            # drop previous stop time column, no longer needed
            trip_summary_stop_service = trip_summary_stop_service.drop('previous_stop_time', axis=1)

            # Remove headway from the first bus in the dataset since we don't have the 
            # previous bus to compare with
            trip_summary_stop_service['scheduled_headway'].iloc[0] = None
            
            # Add
            df_output = pd.concat([df_output, trip_summary_stop_service])



/var/folders/f2/c6nyrb_57g50fl1tj29bgpq40000gn/T/ipykernel_48651/3090683415.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_summary_stop_service.sort_values(by='stop_time',ascending=True, inplace=True)
/var/folders/f2/c6nyrb_57g50fl1tj29bgpq40000gn/T/ipykernel_48651/3090683415.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trip_summary_stop_service['previous_stop_time'] = prev_stop_times
/var/folders/f2/c6nyrb_57g50fl1tj29bgpq40000gn/T/ipykernel_48651/3090683415.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from 

In [ ]:
df_output

## Try it out: Get Scheduled Stop Times and Headways

In [26]:
%%capture --no-display 
# turn off warnings

# Test:  Geet headways for the 55-Garfield bus on 1/9/2023 using the default times (2:30am 1/9 through 2:30am 1/10)
scheduled_headways = get_scheduled_headways('55', '2023-01-09')

In [48]:
### TODO:

# Note that some buses have very long scheduled headways.  Example below - Route 55, stop 10495.
# 55th and Kilpatrick eastbound.  Does this make sense?  It 

scheduled_headways.loc[['stop_id'] == '10495'].sort_values(['stop_id', 'arrival_time']).head(50)

KeyError: 'False: boolean label can not be used without a boolean index'

In [42]:
# Other stops do seem to make sense.  Example: Stop 10603, Garfield and Ashland WB
# on route 55:

scheduled_headways.loc[scheduled_headways['stop_id'] == '10603'].sort_values(['stop_id', 'arrival_time'])


,route_id,service_id,trip_id,direction_id,block_id,shape_id,direction,wheelchair_accessible,schd_trip_id,raw_date,...,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,shape_dist_traveled,arrival_hour,departure_hour,scheduled_headway
985477,55,65301,6530028875020,0,653000003137,65301295,West,1,28875020,2023-01-09 00:00:00+00:00,...,2023-01-09 03:28:00+00:00,03:28:00,10603,1,St Louis,0,0,3,3,NaT
987031,55,65301,6530033660020,0,653000003140,65301295,West,1,33660020,2023-01-09 00:00:00+00:00,...,2023-01-09 03:55:30+00:00,03:55:30,10603,1,St Louis,0,0,3,3,0 days 00:27:30
986920,55,65301,6530033568020,0,653000003141,65301295,West,1,33568020,2023-01-09 00:00:00+00:00,...,2023-01-09 04:18:00+00:00,04:18:00,10603,1,St Louis,0,0,4,4,0 days 00:22:30
975487,55,65301,6530000010020,0,653000003086,65301295,West,1,10020,2023-01-09 00:00:00+00:00,...,2023-01-09 04:37:30+00:00,04:37:30,10603,1,St Louis,0,0,4,4,0 days 00:19:30
983665,55,65301,6530023086020,0,653000003140,65305425,West,1,23086020,2023-01-09 00:00:00+00:00,...,2023-01-09 05:00:37+00:00,05:00:37,10603,35,Midway Orange Line,0,25705,5,5,0 days 00:23:07
976116,55,65301,6530001696020,0,653000003152,65301295,West,1,1696020,2023-01-09 00:00:00+00:00,...,2023-01-09 05:11:30+00:00,05:11:30,10603,1,St Louis,0,0,5,5,0 days 00:10:53
985218,55,65301,6530027998020,0,653000002970,65301295,West,1,27998020,2023-01-09 00:00:00+00:00,...,2023-01-09 05:35:30+00:00,05:35:30,10603,1,St Louis,0,0,5,5,0 days 00:24:00
986476,55,65301,6530032163020,0,653000002965,65301295,West,1,32163020,2023-01-09 00:00:00+00:00,...,2023-01-09 05:57:00+00:00,05:57:00,10603,1,St Louis,0,0,5,5,0 days 00:21:30
988215,55,65301,6530038932020,0,653000003090,65301295,West,1,38932020,2023-01-09 00:00:00+00:00,...,2023-01-09 06:23:00+00:00,06:23:00,10603,1,St Louis,0,0,6,6,0 days 00:26:00
983849,55,65301,6530023418020,0,653000002878,65301295,West,1,23418020,2023-01-09 00:00:00+00:00,...,2023-01-09 06:42:30+00:00,06:42:30,10603,1,St Louis,0,0,6,6,0 days 00:19:30


In [39]:
# Get scheduled headway stats
scheduled_headway_stats = get_headway_stats(scheduled_headways, 'scheduled_headway')

scheduled_headway_stats 

{'mean': Timedelta('0 days 01:33:54.296969696'),
 'max': Timedelta('0 days 16:59:57'),
 'min': Timedelta('0 days 00:01:00'),
 '25th_percentile': Timedelta('0 days 00:12:00'),
 'median': Timedelta('0 days 00:18:00'),
 '75th_percentile': Timedelta('0 days 01:00:00')}

# Get acutal and scheduled headway stats for every stop on a route


In [90]:
# Latest GTFS feed date available from transitfeeds.com
gtfs_version_id = '20230105'

def get_stats_for_stops(
    route_id:str, 
    service_date_string:str, 
    start_timedelta_string:str='02:30', 
    end_timedelta_string:str='26:30') -> pd.DataFrame:

    def get_headway_stats_by_stop(headway_dataframe:pd.DataFrame, stop_id_column:str, headway_column:str) -> pd.DataFrame:

        df_output = pd.DataFrame()

        # df_output['stop_ids'] = scheduled_headways['stop_id'].groupby(by='stop_id')
        df_output[f'{headway_column}_mean'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].mean()
        df_output[f'{headway_column}_min'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].min()
        df_output[f'{headway_column}_25th_percentile'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].quantile(0.25)
        df_output[f'{headway_column}_median'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].median()
        df_output[f'{headway_column}_75th_percentile'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].quantile(.75)
        df_output[f'{headway_column}_max'] = headway_dataframe.groupby(by=stop_id_column)[headway_column].max()

        # move stop id from index to a regular column named 'stop_id'
        df_output.reset_index(drop=True)
        df_output['stop_id'] = df_output.index
        return df_output

    # Scheduled data from gtfs
    gtfs_feed = download_extract_format(gtfs_version_id)

    scheduled_headways = get_scheduled_headways(
        route_id, 
        service_date_string, 
        start_timedelta_string, 
        end_timedelta_string
        )

    df_scheduled_headway_stops = get_headway_stats_by_stop(scheduled_headways,'stop_id', 'scheduled_headway')

    return df_scheduled_headway_stops

    # # Actual data from cta API
    # vehicles = get_chn_vehicles(service_date_string,start_timedelta_string, end_timedelta_string)

    # actual_headways = get_headways(route_id, vehicles)

    # df_actual_headway_stops = get_headway_stats_by_stop(actual_headways, 'stpid', 'est_headway')

    # # Merge scheduled and actual stats
    # output = df_scheduled_headway_stops.merge(df_actual_headway_stops, on='stop_id', how='outer')

    # return output



In [91]:
%%capture --no-display

get_stats_for_stops('55','2023-01-09')

INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
INFO:root:stops.txt loaded
INFO:root:stop_times.txt loaded
INFO:root:routes.txt loaded
INFO:root:trips.txt loaded
INFO:root:calendar.txt loaded
INFO:root:calendar_dates.txt loaded
INFO:root:shapes.txt loaded


,scheduled_headway_mean,scheduled_headway_min,scheduled_headway_25th_percentile,scheduled_headway_median,scheduled_headway_75th_percentile,scheduled_headway_max,stop_id
stop_id,,,,,,,
10495,0 days 02:00:09.833333333,0 days 00:57:00,0 days 00:58:11.250000,0 days 01:02:15,0 days 01:02:26.250000,0 days 04:58:45,10495
10501,NaT,NaT,NaT,NaT,NaT,NaT,10501
10502,NaT,NaT,NaT,NaT,NaT,NaT,10502
10505,NaT,NaT,NaT,NaT,NaT,NaT,10505
10507,0 days 00:52:14.444444444,0 days 00:01:00,0 days 00:18:00,0 days 00:21:31,0 days 00:27:17,0 days 07:58:04,10507
...,...,...,...,...,...,...,...
17563,NaT,NaT,NaT,NaT,NaT,NaT,17563
18018,NaT,NaT,NaT,NaT,NaT,NaT,18018
18032,0 days 05:29:54.500000,0 days 05:00:39,-80064 days +01:24:42.108917760,0 days 05:00:39,-26688 days +03:48:40.036306432,0 days 05:59:10,18032


In [63]:
%%capture --no-display
# turn off warnings

# Scratch
gtfs_version_id = '20230105'
route_id = '55'
service_date_string = '2023-01-09'
start_timedelta_string ='02:30'
end_timedelta_string = '26:30'

gtfs_feed = download_extract_format(gtfs_version_id)



# scheduled_headways = get_scheduled_headways(
#     route_id, 
#     service_date_string, 
#     start_timedelta_string, 
#     end_timedelta_string
#     )


INFO:root:Downloading CTA data
INFO:root:Download complete
INFO:root:Extracting data from CTA zipfile version 20230105
INFO:root:stops.txt loaded
INFO:root:stop_times.txt loaded
INFO:root:routes.txt loaded
INFO:root:trips.txt loaded
INFO:root:calendar.txt loaded
INFO:root:calendar_dates.txt loaded
INFO:root:shapes.txt loaded


In [62]:

df_output = pd.DataFrame()

# df_output['stop_ids'] = scheduled_headways['stop_id'].groupby(by='stop_id')
df_output['sched_mean'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].mean()
df_output['sched_min'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].min()
df_output['sched_25th_percentile'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].quantile(0.25)
df_output['sched_median'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].median()
df_output['sched_75th_percentile'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].quantile(.75)
df_output['sched_max'] = scheduled_headways.groupby(by='stop_id')['scheduled_headway'].max()

df_output




    

,sched_mean,sched_min,sched_25th_percentile,sched_median,sched_75th_percentile,sched_max
stop_id,,,,,,
10495,0 days 02:00:09.833333333,0 days 00:57:00,0 days 00:58:11.250000,0 days 01:02:15,0 days 01:02:26.250000,0 days 04:58:45
10501,NaT,NaT,NaT,NaT,NaT,NaT
10502,NaT,NaT,NaT,NaT,NaT,NaT
10505,NaT,NaT,NaT,NaT,NaT,NaT
10507,0 days 00:52:14.444444444,0 days 00:01:00,0 days 00:18:00,0 days 00:21:31,0 days 00:27:17,0 days 07:58:04
...,...,...,...,...,...,...
17563,NaT,NaT,NaT,NaT,NaT,NaT
18018,NaT,NaT,NaT,NaT,NaT,NaT
18032,0 days 05:29:54.500000,0 days 05:00:39,-80064 days +01:24:42.108917760,0 days 05:00:39,-26688 days +03:48:40.036306432,0 days 05:59:10


In [54]:
scheduled_headways.groupby(by='stop_id')['scheduled_headway'].mean()


quantile(0.25)

stop_id
10495   0 days 02:00:09.833333333
10501                         NaT
10502                         NaT
10505                         NaT
10507   0 days 00:52:14.444444444
                   ...           
17563                         NaT
18018                         NaT
18032      0 days 05:29:54.500000
18448                         NaT
6524              0 days 07:59:52
Name: scheduled_headway, Length: 98, dtype: timedelta64[ns]

In [14]:
### ChatGPT code:

import pandas as pd
import requests

# URL for the GTFS data
url = "http://www.transitchicago.com/downloads/sch_data/google_transit.zip"

# Download the data
response = requests.get(url)
open("google_transit.zip", "wb").write(response.content)

# Load the data into pandas dataframes
dataframes = {
    "routes": pd.read_csv("google_transit/routes.txt"),
    "trips": pd.read_csv("google_transit/trips.txt"),
    "stop_times": pd.read_csv("google_transit/stop_times.txt"),
    "stops": pd.read_csv("google_transit/stops.txt"),
    "calendar": pd.read_csv("google_transit/calendar.txt"),
}

# Print the first 5 rows of the "routes" dataframe
print(dataframes["routes"].head())


FileNotFoundError: [Errno 2] No such file or directory: 'google_transit/routes.txt'

In [16]:
# ChatGPT code
 
import pandas as pd

# Load the GTFS data into a pandas dataframe
stop_times = gtfs_feed.stop_times

# My edit:  convert to timedelta so the code can run
stop_times['arrival_time']=stop_times['arrival_time'].apply(pd.to_timedelta)

# Calculate the time difference between consecutive scheduled arrival times for each trip
stop_times["headway_secs"] = stop_times.sort_values(["trip_id", "stop_sequence"]).groupby("trip_id")["arrival_time"].diff().dt.total_seconds()

# Calculate the average headway for each bus stop
headways = stop_times.groupby("stop_id")["headway_secs"].mean()

# Print the headways in minutes
print(headways / 60)


stop_id
1          0.699632
10         0.379508
100        0.580469
1000       0.397299
10000     -0.097356
            ...    
9996       0.450604
9998       0.356522
9999       0.085172
999990    13.500000
999991     8.500000
Name: headway_secs, Length: 11062, dtype: float64
